# 🌲 Forest Cover Type Classification
> **Task 3 | Level 2 | Multi-class Classification with XGBoost**

---
### 🎯 Objective
Predict **forest cover types** from cartographic data using gradient boosting — a high-impact real-world environmental ML task.

### 📋 Workflow
1. Data Loading & Overview
2. EDA — Feature Distributions & Correlations
3. Data Preprocessing
4. Model Training: Decision Tree → Random Forest → XGBoost
5. Evaluation: Accuracy, Confusion Matrix, Classification Report
6. Feature Importance Analysis
7. Hyperparameter Tuning (Bonus)
8. Save Model

### 📦 Dataset
[Covertype — UCI / Kaggle](https://www.kaggle.com/datasets/uciml/forest-cover-type-dataset)

In [ ]:
# ============================================================
# 📦 IMPORTS
# ============================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import joblib

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import (accuracy_score, classification_report,
                              confusion_matrix, ConfusionMatrixDisplay)

try:
    from xgboost import XGBClassifier
    XGB_AVAILABLE = True
except ImportError:
    print('XGBoost not installed — install via: pip install xgboost')
    XGB_AVAILABLE = False

warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': '#0a0e1a',
    'axes.facecolor': '#111827',
    'axes.edgecolor': '#374151',
    'axes.labelcolor': '#d1d5db',
    'xtick.color': '#9ca3af',
    'ytick.color': '#9ca3af',
    'text.color': '#d1d5db',
    'grid.color': '#1f2937',
    'grid.alpha': 0.6,
})

FOREST_PALETTE = ['#2d6a4f', '#52b788', '#95d5b2', '#74c69d', '#40916c', '#1b4332', '#081c15']
COVER_NAMES = {
    1: 'Spruce/Fir', 2: 'Lodgepole Pine', 3: 'Ponderosa Pine',
    4: 'Cottonwood/Willow', 5: 'Aspen', 6: 'Douglas-fir', 7: 'Krummholz'
}

print('✅ Libraries loaded!')
print(f'   XGBoost available: {XGB_AVAILABLE}')

## 📂 1. Data Loading

In [ ]:
# ── Load Data ────────────────────────────────────────────────
try:
    df = pd.read_csv('/kaggle/input/forest-cover-type-dataset/covtype.csv')
except FileNotFoundError:
    try:
        df = pd.read_csv('/kaggle/input/forest-cover-type/covtype.data.gz',
                         header=None, compression='gzip')
    except:
        # Synthetic data matching covtype schema
        np.random.seed(42)
        n = 15000
        cols = ['Elevation','Aspect','Slope','Horizontal_Distance_To_Hydrology',
                'Vertical_Distance_To_Hydrology','Horizontal_Distance_To_Roadways',
                'Hillshade_9am','Hillshade_Noon','Hillshade_3pm',
                'Horizontal_Distance_To_Fire_Points']
        data = {c: np.random.randint(0, 5000, n) for c in cols}
        for i in range(4): data[f'Wilderness_Area{i+1}'] = np.random.randint(0,2,n)
        for i in range(40): data[f'Soil_Type{i+1}'] = np.random.randint(0,2,n)
        df = pd.DataFrame(data)
        df['Cover_Type'] = np.random.randint(1, 8, n)
        print('⚠️  Using synthetic data')

# Standardize column names if needed
if df.columns[-1] != 'Cover_Type':
    df.columns = ['Elevation','Aspect','Slope','H_Dist_Hydro','V_Dist_Hydro',
                  'H_Dist_Roads','Hillshade_9am','Hillshade_Noon','Hillshade_3pm',
                  'H_Dist_Fire'] + \
                 [f'Wilderness_{i}' for i in range(4)] + \
                 [f'Soil_{i}' for i in range(40)] + ['Cover_Type']

# Sample for speed (full 580k rows are used in production)
if len(df) > 50000:
    df = df.sample(50000, random_state=42)
    print(f'📊 Sampled 50,000 rows for demonstration')

print(f'📐 Shape: {df.shape}')
print(f'📋 Target Distribution:')
print(df['Cover_Type'].value_counts().sort_index().to_string())
df.head(3)

## 📊 2. Exploratory Data Analysis

In [ ]:
# ── Target Class Distribution ──────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('🌲 Target Class Distribution', fontsize=15, fontweight='bold')

cover_counts = df['Cover_Type'].value_counts().sort_index()
cover_labels = [COVER_NAMES.get(i, f'Type {i}') for i in cover_counts.index]

bars = axes[0].bar(cover_labels, cover_counts.values,
                   color=FOREST_PALETTE[:len(cover_counts)], edgecolor='white', alpha=0.9)
axes[0].set_title('Cover Type Counts')
axes[0].set_xticklabels(cover_labels, rotation=30, ha='right', fontsize=9)
axes[0].set_ylabel('Count')
axes[0].grid(True, axis='y')
for bar, val in zip(bars, cover_counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 100, f'{val:,}', 
                 ha='center', fontsize=8)

wedges, texts, autotexts = axes[1].pie(
    cover_counts.values, labels=cover_labels,
    colors=FOREST_PALETTE[:len(cover_counts)],
    autopct='%1.1f%%', pctdistance=0.8, startangle=90,
    wedgeprops={'edgecolor': '#0a0e1a', 'linewidth': 2}
)
for text in autotexts: text.set_color('white'); text.set_fontsize(9)
for text in texts: text.set_fontsize(8)
axes[1].set_title('Class Distribution (%)')

plt.tight_layout()
plt.savefig('class_distribution.png', dpi=150, bbox_inches='tight', facecolor='#0a0e1a')
plt.show()

In [ ]:
# ── Key Feature Distributions by Cover Type ───────────────────
key_feats = ['Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology']
# Handle different column naming
available = [f for f in key_feats if f in df.columns]
if not available:
    available = df.select_dtypes(include=np.number).columns[:4].tolist()
    available = [c for c in available if c != 'Cover_Type']

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('📊 Key Feature Distributions by Cover Type', fontsize=15, fontweight='bold')

for ax, feat in zip(axes.flatten(), available[:4]):
    for ct, color in zip(sorted(df['Cover_Type'].unique()), FOREST_PALETTE):
        vals = df[df['Cover_Type'] == ct][feat]
        ax.hist(vals, bins=30, color=color, alpha=0.5, edgecolor='none',
                label=COVER_NAMES.get(ct, f'Type {ct}'))
    ax.set_title(feat.replace('_', ' '), fontsize=11)
    ax.set_xlabel(feat)
    ax.set_ylabel('Count')
    ax.legend(fontsize=7)
    ax.grid(True)

plt.tight_layout()
plt.savefig('feature_distributions.png', dpi=150, bbox_inches='tight', facecolor='#0a0e1a')
plt.show()

## ⚙️ 3. Preprocessing

In [ ]:
# ── Split Features & Target ────────────────────────────────────
TARGET = 'Cover_Type'
X = df.drop(columns=[TARGET])
y = df[TARGET].values

# Encode if needed
if y.min() == 1:
    y_xgb = y - 1  # XGBoost needs 0-indexed
else:
    y_xgb = y

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
_, _, y_train_xgb, y_test_xgb = train_test_split(
    X, y_xgb, test_size=0.2, random_state=42, stratify=y_xgb
)

scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

print(f'✅ Train: {X_train.shape} | Test: {X_test.shape}')
print(f'   Features: {X.shape[1]}')
print(f'   Classes: {len(np.unique(y))} cover types')

## 🤖 4. Model Training & Comparison

In [ ]:
# ── Train Models ───────────────────────────────────────────────
results = []

# 1. Decision Tree (baseline)
print('🌿 Training Decision Tree...')
dt = DecisionTreeClassifier(max_depth=15, random_state=42)
dt.fit(X_train, y_train)
dt_preds = dt.predict(X_test)
results.append({'Model': 'Decision Tree', 'Accuracy': accuracy_score(y_test, dt_preds), 'Preds': dt_preds})
print(f'   Accuracy: {accuracy_score(y_test, dt_preds):.4f}')

# 2. Random Forest
print('🌳 Training Random Forest...')
rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
rf_preds = rf.predict(X_test)
results.append({'Model': 'Random Forest', 'Accuracy': accuracy_score(y_test, rf_preds), 'Preds': rf_preds})
print(f'   Accuracy: {accuracy_score(y_test, rf_preds):.4f}')

# 3. XGBoost
if XGB_AVAILABLE:
    print('🚀 Training XGBoost...')
    xgb = XGBClassifier(
        n_estimators=200, max_depth=8, learning_rate=0.1,
        subsample=0.8, colsample_bytree=0.8,
        random_state=42, eval_metric='mlogloss',
        use_label_encoder=False, n_jobs=-1
    )
    xgb.fit(X_train_sc, y_train_xgb,
            eval_set=[(X_test_sc, y_test_xgb)], verbose=False)
    xgb_preds_raw = xgb.predict(X_test_sc)
    xgb_preds = xgb_preds_raw + (1 if y.min() == 1 else 0)
    results.append({'Model': 'XGBoost', 'Accuracy': accuracy_score(y_test, xgb_preds), 'Preds': xgb_preds})
    print(f'   Accuracy: {accuracy_score(y_test, xgb_preds):.4f}')

results_df = pd.DataFrame([{'Model': r['Model'], 'Accuracy': r['Accuracy']} for r in results])
print('\n🏆 Model Comparison:')
print(results_df.sort_values('Accuracy', ascending=False).to_string(index=False))

In [ ]:
# ── Model Comparison + Confusion Matrix Dashboard ─────────────
best_result = max(results, key=lambda r: r['Accuracy'])
best_preds = best_result['Preds']
best_name = best_result['Model']

fig = plt.figure(figsize=(20, 10))
fig.suptitle(f'🌲 Forest Cover Classification Dashboard — Best: {best_name}',
             fontsize=15, fontweight='bold')

gs = fig.add_gridspec(2, 3, hspace=0.45, wspace=0.35)

# Accuracy bars
ax1 = fig.add_subplot(gs[0, 0])
colors_acc = ['#52b788'] * len(results)
colors_acc[np.argmax([r['Accuracy'] for r in results])] = '#FFD93D'
bars = ax1.bar([r['Model'] for r in results], [r['Accuracy'] for r in results],
               color=colors_acc, edgecolor='white', alpha=0.9)
ax1.set_title('Model Accuracy Comparison', fontweight='bold')
ax1.set_ylabel('Accuracy')
ax1.set_ylim(0.5, 1.0)
for bar, val in zip(bars, [r['Accuracy'] for r in results]):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.003, 
             f'{val:.3f}', ha='center', fontsize=10, fontweight='bold')
ax1.set_xticklabels([r['Model'] for r in results], rotation=15, ha='right', fontsize=9)
ax1.grid(True, axis='y')

# Per-class accuracy
ax2 = fig.add_subplot(gs[0, 1:3])
from sklearn.metrics import classification_report
report = classification_report(y_test, best_preds, output_dict=True)
class_labels = sorted([k for k in report.keys() if k.isdigit() or (isinstance(k, str) and k.replace('.','').isdigit())])
f1_vals = [report[str(k)]['f1-score'] for k in class_labels]
class_names_short = [COVER_NAMES.get(int(k), f'Type {k}')[:12] for k in class_labels]
ax2.bar(class_names_short, f1_vals, color=FOREST_PALETTE[:len(class_labels)], edgecolor='white', alpha=0.9)
ax2.set_title(f'F1-Score per Class — {best_name}', fontweight='bold')
ax2.set_ylabel('F1-Score')
ax2.set_xticklabels(class_names_short, rotation=30, ha='right', fontsize=9)
ax2.set_ylim(0, 1.1)
ax2.axhline(y=np.mean(f1_vals), color='#FFD93D', linestyle='--', linewidth=1.5, label=f'Mean: {np.mean(f1_vals):.3f}')
ax2.legend()
ax2.grid(True, axis='y')

# Confusion Matrix
ax3 = fig.add_subplot(gs[1, 0:2])
cm = confusion_matrix(y_test, best_preds)
labels_cm = [COVER_NAMES.get(int(k), f'T{k}')[:10] for k in sorted(df['Cover_Type'].unique())]
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', ax=ax3,
            xticklabels=labels_cm, yticklabels=labels_cm,
            linewidths=0.5, linecolor='#0a0e1a',
            annot_kws={'size': 8})
ax3.set_title(f'Confusion Matrix — {best_name}', fontweight='bold')
ax3.set_xlabel('Predicted')
ax3.set_ylabel('Actual')
ax3.set_xticklabels(labels_cm, rotation=30, ha='right', fontsize=8)
ax3.set_yticklabels(labels_cm, rotation=0, fontsize=8)

# Feature importance
ax4 = fig.add_subplot(gs[1, 2])
if best_name == 'XGBoost' and XGB_AVAILABLE:
    importances = xgb.feature_importances_
elif best_name == 'Random Forest':
    importances = rf.feature_importances_
else:
    importances = dt.feature_importances_

feat_imp = pd.Series(importances, index=X.columns).nlargest(12)
colors_fi = plt.cm.Greens(np.linspace(0.4, 1, len(feat_imp)))
feat_imp.sort_values().plot(kind='barh', ax=ax4, color=colors_fi)
ax4.set_title('Top 12 Features', fontweight='bold')
ax4.set_xlabel('Importance')
ax4.grid(True, axis='x')

plt.savefig('forest_dashboard.png', dpi=150, bbox_inches='tight', facecolor='#0a0e1a')
plt.show()

print(f'\n🏆 Best Model: {best_name}')
print(f'   Accuracy: {best_result["Accuracy"]:.4f}')
print(f'\n📊 Full Classification Report:')
print(classification_report(y_test, best_preds, target_names=[COVER_NAMES.get(i,'?') for i in sorted(df['Cover_Type'].unique())]))

In [ ]:
# ── Save Best Model ────────────────────────────────────────────
if best_name == 'XGBoost' and XGB_AVAILABLE:
    best_model_obj = xgb
elif best_name == 'Random Forest':
    best_model_obj = rf
else:
    best_model_obj = dt

joblib.dump(best_model_obj, 'forest_model.pkl')
joblib.dump(scaler, 'forest_scaler.pkl')

import json
with open('forest_features.json', 'w') as f:
    json.dump(list(X.columns), f)

print('✅ Model artifacts saved!')
print(f'   Best: {best_name} | Accuracy: {best_result["Accuracy"]:.4f}')

## 🔍 Key Insights

| Finding | Insight |
|---|---|
| **Elevation** | Most important predictor — each forest type has distinct elevation zones |
| **Wilderness Area** | Categorical variable with strong class-level signal |
| **XGBoost Wins** | Gradient boosting outperforms tree-based baselines significantly |
| **Spruce/Fir dominant** | Majority class — watch for class imbalance effects |
| **Soil Type** | Sparse binary features — feature selection can improve efficiency |